<a href="https://colab.research.google.com/github/tomheston/fragility-metrics/blob/main/notebooks/Meaningful_Change_Index.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# @title MeCI (Meaningful Change Index) — Colab-ready single cell
# Definition: MeCI = min(|mean1 - c|, |mean2 - c|) / sqrt(s1^2 + s2^2)
# where c = (s1*mean2 + s2*mean1) / (s1 + s2)
# Source: FRAGILITY METRICS LOCKED IN DEFINITIONS v1.3 (2025-10-10)

import math

def compute_crossover(mean1: float, s1: float, mean2: float, s2: float) -> float:
    if s1 <= 0 or s2 <= 0:
        raise ValueError("Standard deviations must be > 0.")
    return (s1 * mean2 + s2 * mean1) / (s1 + s2)

def compute_meci(mean1: float, s1: float, mean2: float, s2: float):
    c = compute_crossover(mean1, s1, mean2, s2)
    num = min(abs(mean1 - c), abs(mean2 - c))
    den = math.sqrt(s1**2 + s2**2)
    meci = num / den
    return c, num, den, meci

def interpret_meci(meci: float) -> str:
    if meci < 0.20: return "High overlap, very fragile."
    if meci < 0.50: return "Moderate overlap, some fragility."
    if meci <= 1.00: return "Low overlap, reasonably robust."
    return "Minimal overlap, strong separation."

def prompt_float(label: str) -> float:
    while True:
        try:
            return float(input(f"{label}: ").strip())
        except Exception as e:
            print(f"Invalid number. Try again. ({e})")

# --- Run interactively on cell execution ---
print("Enter group summary statistics (means and standard deviations).")
m1 = prompt_float("mean1")
s1 = prompt_float("std1")
m2 = prompt_float("mean2")
s2 = prompt_float("std2")

try:
    c, num, den, meci = compute_meci(m1, s1, m2, s2)
    print("\n--- Meaningful Change Index (MeCI) ---")
    print(f"crossover c = {c:.6f}")
    print(f"numerator   = {num:.6f}   (min distance to c)")
    print(f"denominator = {den:.6f}   (sqrt(s1^2 + s2^2))")
    print(f"MeCI        = {meci:.6f}")
    print(f"Interpretation: {interpret_meci(meci)}")
    print(f"  ")
    print(f"MeCI fragility categories:")
    print(f" < 0.20 = High overlap, very fragile.")
    print(f" < 0.50 = Moderate overlap, some fragility.")
    print(f" <= 1.00 = Low overlap, reasonably robust.")
    print(f" > 1.00 = Minimal overlap, strong separation.")
    if abs(s1 - s2) <= 1e-9:
        d = (m1 - m2) / s1
        print(f"\nEqual-variance note: Cohen's d = {d:.6f}; expected MeCI ≈ {abs(d)/(2*math.sqrt(2)):.6f}")
except Exception as e:
    print(f"\nError: {e}")


Enter group summary statistics (means and standard deviations).
mean1: 32
std1: 1.1
mean2: 44
std2: 7.8

--- Meaningful Change Index (MeCI) ---
crossover c = 33.483146
numerator   = 1.483146   (min distance to c)
denominator = 7.877182   (sqrt(s1^2 + s2^2))
MeCI        = 0.188284
Interpretation: High overlap, very fragile.
  
MeCI fragility categories:
 < 0.20 = High overlap, very fragile.
 < 0.50 = Moderate overlap, some fragility.
 <= 1.00 = Low overlap, reasonably robust.
 > 1.00 = Minimal overlap, strong separation.
